In [3]:
# 주피터
# import os

# os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

In [2]:
# 코랩
from google.colab import drive
drive.mount("/content/drive")

import os
path = '/content/drive/My Drive/projects/NLP-StockMarket/model_fin/' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

Mounted at /content/drive


## 필요모듈 import

In [ ]:
!pip install konlpy   # 코랩에서만 실행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
from tqdm import tqdm
import scipy as sp
import pandas as pd
import numpy as np
import re

from konlpy.tag import Kkma    
from konlpy.tag import Hannanum    
from konlpy.tag import Okt        
from konlpy.tag import *
import nltk
import pickle

# from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz
from datetime import datetime, timedelta

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
from keras.preprocessing.text import Tokenizer


import warnings
warnings.filterwarnings('ignore')

In [2]:
kkma = Kkma()
hannanum = Hannanum()
okt = Okt()

## 자연어 데이터 import(날짜 가져오기 위해 먼저 가져옴)

In [3]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [4]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col=0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col=0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col=0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col=0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col=0)

In [5]:
naver_talks['Date'] = pd.to_datetime(naver_talks['Date'])
daum_talks['Date'] = pd.to_datetime(daum_talks['Date'])

In [6]:
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))     # str(변수) 는 자체 내장함수, 변수 하나씩밖에 적용 안됨.
news_df.sort_values('Date', ignore_index=True, inplace=True)

In [7]:
news_df[news_df['Date']== '2021-01-04'].head(2)

,Date,Title
639,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요
640,2021-01-04,마감 국채선물 상승 111 52 증가5틱


In [8]:
news_df[news_df['Date']== '2021-01-04'].index[0]

639

In [9]:
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0]:]
news_df.head(3)

,Date,Title
639,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요
640,2021-01-04,마감 국채선물 상승 111 52 증가5틱
641,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기


## 주가 데이터

### KRX에서 받은 csv로 가져오기

In [10]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head()

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24
2,2021-01-06,-0.87
3,2021-01-07,2.65
4,2021-01-08,5.87


In [11]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


## 합치기

In [12]:
stock_df['updown'] = 0

In [13]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

In [14]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률', 'updown']
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
print(len(df))
df.head()

429597


,Date,Title,주가의 날짜,등락률,updown
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,0
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,0
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,0
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,0
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,0


In [15]:
sentiment_csv = pd.read_csv('./sentiment dictionary new.csv', index_col = 0)
sentiment_csv.head()

,Pos,Neg,Mid
0,경신,매도,책임
1,상승세,조정,랠리
2,오른,감소,유지
3,강화,전쟁,중립
4,상승,개발,산타랠리


In [16]:
pos_li = sentiment_csv['Pos'].dropna().values
mid_li = sentiment_csv['Mid'].dropna().values
neg_li = sentiment_csv['Neg'].dropna().values

In [19]:
### binary_sentimental_score
def binary_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['감성지수'] = 0
    
    # SP 등락률에 따른 updown 계산
    df_result.loc[df_result.query('등락률 >= 0').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 < 0').index, 'updown'] = -1

    # 감성 지수는 긍정 : 1, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999    
    
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1').index, '감성지수'] = -1
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1').index].index) : 
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]
        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 0 :
            df_result.loc[i,'감성지수'] = 1
        else :
            df_result.loc[i,'감성지수'] = -1
    df_result = pd.concat([df_result.loc[df_result.query('감성지수 != 999').index]], ignore_index = True)
    return df_result

In [20]:
### multi_sentimental_score
def multi_sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['Mid'] = 0
    df_result['감성지수'] = 0
    
    # SP 등락률에 따른 updown 계산
    df_result.loc[df_result.query('등락률 > 1').index, 'updown'] = 1
    df_result.loc[df_result.query('등락률 < -1').index, 'updown'] = -1
    
    # 감성 지수는 긍정 : 1, 중립 : 0, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999    
    
################################################################################################
    # 0  : 없음, 1: 있음
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"Title.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"Title.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    print('중립 단어 검색중')
    for mid in tqdm(mid_li) :
        str_expr = f"Title.str.contains('{mid}')"
        df_result.loc[df_result.query(str_expr).index, 'Mid'] = 1
    
    
################################################################################################
    
    # 모든 종류의 단어가 검출 되면 제외
    df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 1').index, '감성지수'] = 999
    
    # 중립 단어가 검출되면 중립
    df_result.loc[df_result.query('Mid == 1').index, '감성지수'] = 0
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0 and Mid == 0').index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1 and Mid == 0').index, '감성지수'] = -1
    
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 0').index].index) : 
        
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        if sum(df_result.loc[i,'Date'] - timedelta(days = 1) == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = 1)]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 :  
            updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date']]['등락률'].values[0]
        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                if sum(df_result.loc[i,'Date'] - timedelta(days = j) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == df_result.loc[i,'Date'] - timedelta(days = j)]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 1 :
            df_result.loc[i,'감성지수'] = 1
        elif updown < -1 :
            df_result.loc[i,'감성지수'] = -1
        else :
            df_result.loc[i,'감성지수'] = 0
    df_result = pd.concat([df_result.loc[df_result.query('감성지수 != 999').index]], ignore_index = True)
    return df_result

In [21]:
df_result = multi_sentimental_score(df)
df_result

긍정 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [12:26<00:00,  1.34it/s]


부정 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████| 1323/1323 [15:25<00:00,  1.43it/s]


중립 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [00:58<00:00,  1.46it/s]


긍정 부정 둘 다 있는 경우 처리중


100%|███████████████████████████████████████████████████████████████████████████| 84358/84358 [03:55<00:00, 358.27it/s]


,Date,Title,주가의 날짜,등락률,updown,Pos,Neg,Mid,감성지수
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,1,1,0,0,1
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,1,1,0,0,1
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,1,1,0,0,1
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,1,1,0,0,1
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...
364467,2022-06-29,수젠텍 원숭이두창 분자진단제품 개발 중 분자진단 플렛폼기술 확보,2022-06-30,-6.67,-1,1,1,0,1
364468,2022-06-29,LS 주가 하락장서 역주행 모든 사업부가 호황,2022-06-30,-6.67,-1,1,1,0,1
364469,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,-1,1,0,0,1
364470,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,-1,0,1,0,-1


In [22]:
df_result

,Date,Title,주가의 날짜,등락률,updown,Pos,Neg,Mid,감성지수
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,1,1,0,0,1
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,1,1,0,0,1
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,1,1,0,0,1
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,1,1,0,0,1
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...
364467,2022-06-29,수젠텍 원숭이두창 분자진단제품 개발 중 분자진단 플렛폼기술 확보,2022-06-30,-6.67,-1,1,1,0,1
364468,2022-06-29,LS 주가 하락장서 역주행 모든 사업부가 호황,2022-06-30,-6.67,-1,1,1,0,1
364469,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,-1,1,0,0,1
364470,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,-1,0,1,0,-1


In [23]:
df_result.to_csv('sentiment_analysis_result_data_for_model8.csv', encoding='utf-8-sig')

## 감성분석 모델 적용

In [41]:
df = pd.read_csv('./sentiment_analysis_result_data_for_model8.csv', index_col=0)
df

,Date,Title,주가의 날짜,등락률,updown,Pos,Neg,Mid,감성지수
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,1,1,0,0,1
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,1,1,0,0,1
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,1,1,0,0,1
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,1,1,0,0,1
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...
364467,2022-06-29,수젠텍 원숭이두창 분자진단제품 개발 중 분자진단 플렛폼기술 확보,2022-06-30,-6.67,-1,1,1,0,1
364468,2022-06-29,LS 주가 하락장서 역주행 모든 사업부가 호황,2022-06-30,-6.67,-1,1,1,0,1
364469,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,-1,1,0,0,1
364470,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,-1,0,1,0,-1


In [42]:
df['updown'].value_counts()

 0    143890
 1    120122
-1    100460
Name: updown, dtype: int64

In [43]:
df['감성지수'].value_counts()

 1    211270
-1     89372
 0     63830
Name: 감성지수, dtype: int64

In [48]:
n_ = []
title_rename = []
for i in tqdm(range(len(df))):
    title_rename.append(re.sub("[\(\[].*?[\)\]]", "",df.iloc[i]['Title']))
    n_.append(' '.join(okt.nouns(df.iloc[i]['Title'])))
df['nouns'] = n_
df['Title']=title_rename
df = df[df['nouns']!='']

100%|█████████████████████████████████████████████████████████████████████████| 364472/364472 [23:02<00:00, 263.69it/s]


In [51]:
vocab = {}
cnt = 0
for i in tqdm(df['nouns']):
    i = i.split(' ')
    for j in range(len(i)):
        if i[j] in vocab or len(i[j]) <= 1:
            cnt = cnt+1
            pass
        else:
            vocab[i[j]] = 0
print(len(vocab))
print(vocab)

100%|██████████████████████████████████████████████████████████████████████| 364333/364333 [00:02<00:00, 140764.72it/s]

31170
{'영채': 0, '대표': 0, '고객': 0, '신뢰': 0, '중요': 0, '마감': 0, '국채선물': 0, '상승': 0, '증가': 0, '새해': 0, '첫날': 0, '주식시장': 0, '축포': 0, '돌파': 0, '초읽기': 0, '프로': 0, '스테': 0, '믹스': 0, '시설': 0, '증설': 0, '투자': 0, '프로그램': 0, '매도': 0, '우위': 0, '거래': 0, '코스피': 0, '증권': 0, '간다': 0, '속보': 0, '증시': 0, '최고': 0, '장원': 0, '테크': 0, '규모': 0, '전환사채': 0, '전환': 0, '가액': 0, '조정': 0, '지수선물': 0, '사상': 0, '행진': 0, '신년사': 0, '이영창': 0, '금투': 0, '사장': 0, '올해': 0, '변곡점': 0, '혁신': 0, '마지막': 0, '기회': 0, '연초': 0, '동학': 0, '개미': 0, '특징': 0, '면역': 0, '항암제': 0, '개발': 0, '보합': 0, '하락': 0, '혼자': 0, '레벨': 0, '디앤': 0, '씨미디어': 0, '수출': 0, '급증': 0, '주가': 0, '급등': 0, '경신': 0, '수익률': 0, '랠리': 0, '계속': 0, '극찬': 0, '서학': 0, '유치': 0, '전쟁': 0, '해외': 0, '계좌': 0, '가입': 0, '서현': 0, '에너지': 0, '전문가': 0, '성오': 0, '본부장': 0, '에프': 0, '감소': 0, '바이오': 0, '보툴리눔': 0, '톡신': 0, '공장': 0, '설립': 0, '글로벌': 0, '시장': 0, '공략': 0, '진천': 0, '증축': 0, '계약': 0, '보고': 0, '스톡': 0, '톡스': 0, '산업': 0, '변환기': 0, '구조': 0, '저하': 0, '전망': 0, '화승': 0, '알앤': 0, '에이': 0, '신

In [52]:
# vocab_freq.csv 저장!!
import csv

# dct = {'Name': 'John', 'Age': '23', 'Country': 'USA'}

with open('[Model2]vocab_freq.csv', 'w', newline = '', encoding = 'utf-8-sig') as f:  
    writer = csv.writer(f)
    for k, v in vocab.items():
        writer.writerow([k, v])

### binary

In [33]:
### binary
up = df['감성지수'].value_counts()[1]
down = df['감성지수'].value_counts()[0]
up_ratio = up/(up+down)
down_ratio = down/(up+down)

import collections
for i,w in tqdm(enumerate(df['nouns'])):
    w = w.split(' ')
    if (df.iloc[i]['감성지수']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
                continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
                continue
            vocab[noun] = vocab[noun] - up_ratio

364333it [01:26, 4194.43it/s]


In [34]:
# # vocab_score 저장!!!
# with open('[Model8] vocab_score.csv', 'w', newline = '', encoding = 'utf-8-sig') as f:  
#     writer = csv.writer(f)
#     for k, v in vocab.items():
#        writer.writerow([k, v])

In [35]:
sent_dictionary = vocab
total = []
for i,w  in tqdm(enumerate(df['nouns'])):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
            continue
        elif(j not in sent_dictionary):
            continue
        else:
            sent_score = sent_score + sent_dictionary[j]
    total.append(sent_score/len(w))
df['sent_score'] = total

364333it [00:05, 64763.48it/s]


In [36]:
## 평균값 이상인 경우 1, 아닌 경우 0
df['sent_label'] = 0
df

,Date,Title,주가의 날짜,등락률,updown,Pos,Neg,Mid,감성지수,nouns,sent_score,sent_label
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,1,1,0,0,1,정 영채 대표 고객 신뢰 중요,-110.182936,0
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,1,1,0,0,1,마감 국채선물 상승 증가,-2072.636422,0
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,1,1,0,0,1,새해 첫날 주식시장 축포 돌파 초읽기,101.621105,0
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,1,1,0,0,1,프로 스테 믹스 시설 증설 투자,-253.157015,0
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,1,1,1,0,1,마감 프로그램 매도 우위,-4687.534820,0
...,...,...,...,...,...,...,...,...,...,...,...,...
364467,2022-06-29,수젠텍 원숭이두창 분자진단제품 개발 중 분자진단 플렛폼기술 확보,2022-06-30,-6.67,-1,1,1,0,1,수젠텍 원숭이 두창 분자 진단 제품 개발 중 분자 진단 플렛 폼 기술 확보,-612.281888,0
364468,2022-06-29,LS 주가 하락장서 역주행 모든 사업부가 호황,2022-06-30,-6.67,-1,1,1,0,1,주가 하락 장서 역주행 모든 사업 부가 호황,-5804.924109,0
364469,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,-1,1,0,0,1,바사 토종 백신 호 식약처 승인 소식 강세,-79.121030,0
364470,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,-1,0,1,0,-1,장 황 코스피 선 경기 침체 우려 약세,-3643.460386,0


In [37]:
sum = 0
for i in tqdm(range(len(vocab))):
    sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in tqdm(range(len(df))):
    if(df.iloc[i]['sent_score']>sent_mean):
        a_.append(1)
    else:
        a_.append(0)

df['sent_label'] = a_
print("sent_mean :", sent_mean)
df.head(5)

100%|████████████████████████████████████████████████████████████████████████| 364333/364333 [01:22<00:00, 4408.23it/s]


sent_mean : -29.47454411649045


,Date,Title,주가의 날짜,등락률,updown,Pos,Neg,Mid,감성지수,nouns,sent_score,sent_label
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,1,1,0,0,1,정 영채 대표 고객 신뢰 중요,-110.182936,0
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,1,1,0,0,1,마감 국채선물 상승 증가,-2072.636422,0
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,1,1,0,0,1,새해 첫날 주식시장 축포 돌파 초읽기,101.621105,1
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,1,1,0,0,1,프로 스테 믹스 시설 증설 투자,-253.157015,0
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,1,1,1,0,1,마감 프로그램 매도 우위,-4687.534820,0


In [38]:
df['sent_label'].value_counts()

0    343903
1     20430
Name: sent_label, dtype: int64

In [ ]:
# Mecab 은 colab!

In [39]:
df['tokenized'] = 0

In [40]:
# from konlpy.tag import Mecab
# train_data, validation_data = train_test_split(df_train, test_size = 0.2, random_state = 42)

df.dropna(how='any',inplace= True)
# validation_data.dropna(how='any', inplace=True)
df['tokenized'] = df['Title'].apply(okt.morphs)   # 또는 자유롭게 라이브러리 사용
# mecab = Mecab() 
# validation_data['tokenized'] = validation_data['제목'].apply(mecab.morphs)
X_train = df['tokenized'].values
Y_train = df['sent_label'].values
# X_vali= validation_data['tokenized'].values
# Y_vali = validation_data['sent_label'].values

KeyboardInterrupt: 

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# 빈도 수가 낮은 단어 제거
threshold = 3 # 3개 미만이면 제거

words_cnt = len(tokenizer.word_index)
rare_cnt = 0
words_freq = 0
rare_freq = 0

for key, value in tqdm(tokenizer.word_counts.items()):
    words_freq = words_freq + value

    if value < threshold:
        rare_cnt += 1
        rare_freq = rare_freq + value

print("전체 단어 수 :", words_cnt)
print("빈도가 {} 이하인 희귀 단어 수 : {}".format(threshold-1, rare_cnt))
print("희귀 단어 비율 : {}".format((rare_cnt/words_cnt) * 100))
print("희귀 단어 등장 빈도 비율 : {}".format((rare_freq / words_freq) * 100))

In [ ]:
vocab_size = words_cnt - rare_cnt + 2
print(vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
# X_vali = tokenizer.texts_to_sequences(X_vali)

In [ ]:
print('리뷰 최대 길이 :', max(len(i) for i in X_train))
# print('리뷰 평균 길이 :', sum(map(len, X_train))/ len(X_train))

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in tqdm(nested_list):
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = max(len(i) for i in X_train)
below_threshold_len(max_len, X_train)

# 패딩
X_train = pad_sequences(X_train, maxlen = max_len)
# X_vali = pad_sequences(X_vali, maxlen = max_len)

#### 모델링-LSTM

In [94]:
from tensorflow.keras.layers import Embedding, Dense, GRU, LSTM, Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', 
#                      monitor='val_acc', 
                     mode='max', verbose=1, save_best_only=True)

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])    # multinominal
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])  # binary
history = model.fit(X_train, Y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

model.summary()

Epoch 1/15


ValueError: in user code:

    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = l

In [91]:
Y_result = df['updown'].values

In [93]:
# 저장된 모델 불러오기, 평가
loaded_model = load_model('best_model.h5')
loaded_model.evaluate(X_train, Y_result)

11386/11386 [==============================] - 83s 7ms/step - loss: 3.5623 - acc: 0.3650


[3.5623388290405273, 0.3650369346141815]

In [2]:
# # 시각화

# hist_dict = history.history
# loss = hist_dict['loss']
# val_loss = hist_dict['val_loss']
# acc = hist_dict['acc']
# val_acc = hist_dict['val_acc']

# plt.plot(loss, 'b--', label = 'training loss')
# plt.plot(val_loss, 'r--', label = 'validation loss')
# plt.legend()
# plt.grid()

# plt.figure()
# plt.plot(acc, 'b--', label = 'training accuracy')
# plt.plot(val_acc, 'r--', label = 'validation accuracy')
# plt.legend()
# plt.grid()

# plt.show()

In [47]:
# 임의 불용어
stopwords1 = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords2 = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
stopwords = list(set(stopwords1 + stopwords2))

def sentiment_predict(new_sentence):
    new_token = [word for word in okt.morphs(new_sentence) if not word in stopwords]
    new_sequences = tokenizer.texts_to_sequences([new_token])
    new_pad = pad_sequences(new_sequences, maxlen = max_len)
    score = float(loaded_model.predict(new_pad))

    if score > 0.5:
        print("{} -> 긍정 ({:.2f}%)".format(new_sentence, score * 100))
    else:
        print("{} -> 부정 ({:.2f}%)".format(new_sentence, (1-score) * 100))

In [48]:
sentiment_predict("[유럽개장] 장초반 상승세…英 1.03%↑")
sentiment_predict("[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감")

[유럽개장] 장초반 상승세…英 1.03%↑ -> 긍정 (100.00%)
[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감 -> 부정 (100.00%)


### multi

In [53]:
up = len(df[df['감성지수'] == 1])
down = len(df[df['감성지수'] == -1])
up_ratio = up/(up + down)
down_ratio = down/(up + down)

print('up_ratio : ', up_ratio)
print('down_ratio : ', down_ratio)

for i, w in tqdm(enumerate(df['nouns'])) : 
    w = w.split(' ') # 공백 기준으로 분리한 기사 제목 단어들의 리스트

    if df.iloc[i]['감성지수'] == 1 : # 등락률이 양수면 
        for j in range(len(w)) : # 리스트의 길이만큼 반복문을 돌려서
            noun = w[j] # 단어를 뽑아내고
            if len(noun) <=1 : # 단어가 없거나 1개 뿐이라면
                continue
            vocab[noun] += down_ratio # vocab의 noun키에 값 할당
            
    elif df.iloc[i]['감성지수'] == 0 :   # 등락률이 0이면
        for j in range(len(w)) : # 리스트의 길이만큼 반복문을 돌려서
            noun = w[j] # 단어를 뽑아내고
            if len(noun) <=1 : # 단어가 없거나 1개 뿐이라면
                continue
            vocab[noun] = vocab[noun]

    elif df.iloc[i]['감성지수'] == -1 : 
        for j in range(len(w)) : 
            noun = w[j]
            if len(noun) <= 1:
                continue
            vocab[noun] -= up_ratio # vocab의 noun키에 값 할당

up_ratio :  0.7027703366000566
down_ratio :  0.29722966339994344


364333it [02:14, 2709.98it/s]


In [ ]:
# # 단어 점수 csv 저장

# with open('./data/[Model4]vocab_score.csv', 'w', newline='', encoding = 'utf-8-sig') as f:
#     w = csv.writer(f)
#     w.writerow(vocab.keys())
#     w.writerow(vocab.values())

In [54]:
sent_dictionary = vocab
total = []

for i, w in tqdm(enumerate(df['nouns'])):
    sent_score = 0
    w = w.split(' ')

    for j in w : 
        if len(j) <= 1 :
            continue
        elif j not in sent_dictionary : 
            continue
        else : 
            sent_score = sent_score + sent_dictionary[j]

    total.append(sent_score/len(w))
df['sent_score'] = total

364333it [00:03, 106352.38it/s]


In [55]:
# 평균값 초과인 경우 1, 비슷한 경우 0, 미만인 경우 -1
df['sent_label'] = 0

In [57]:
sums = pd.DataFrame(vocab.values()).sum()[0]
sent_mean = sums/len(vocab.keys())
print('sent_mean : ', sent_mean)

sent_mean :  0.3879093539772399


In [58]:
df['sent_score'].describe()

count    364333.000000
mean        -46.641585
std        1064.349745
min      -12788.110947
25%        -209.605544
50%          79.879445
75%         415.603017
max        8111.940766
Name: sent_score, dtype: float64

In [59]:
a_ = []
for i in tqdm(range(len(df))) :  
    if df.iloc[i]['sent_score'] > 5 :    # ========================================================================숫자 확정 필요!!!
        a_.append(1)
    elif df.iloc[i]['sent_score'] < sent_mean :
        a_.append(-1)
    else :
        a_.append(0)
df['sent_label'] = a_
df

100%|████████████████████████████████████████████████████████████████████████| 364333/364333 [01:50<00:00, 3300.01it/s]


,Date,Title,주가의 날짜,등락률,updown,Pos,Neg,Mid,감성지수,nouns,sent_score,sent_label
0,2021-01-04,정영채 NH證 대표 고객과의 신뢰 중요,2021-01-05,2.24,1,1,0,0,1,정 영채 대표 고객 신뢰 중요,52.637560,1
1,2021-01-04,마감 국채선물 상승 111 52 증가5틱,2021-01-05,2.24,1,1,0,0,1,마감 국채선물 상승 증가,2113.599047,1
2,2021-01-04,새해 첫날 주식시장 축포 2900 돌파 3000 초읽기,2021-01-05,2.24,1,1,0,0,1,새해 첫날 주식시장 축포 돌파 초읽기,265.428439,1
3,2021-01-04,프로스테믹스 시설 증설에 55억 투자,2021-01-05,2.24,1,1,0,0,1,프로 스테 믹스 시설 증설 투자,701.723391,1
4,2021-01-04,마감 프로그램 5841억 매도우위,2021-01-05,2.24,1,1,1,0,1,마감 프로그램 매도 우위,-1310.488684,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
364467,2022-06-29,수젠텍 원숭이두창 분자진단제품 개발 중 분자진단 플렛폼기술 확보,2022-06-30,-6.67,-1,1,1,0,1,수젠텍 원숭이 두창 분자 진단 제품 개발 중 분자 진단 플렛 폼 기술 확보,-54.156842,-1
364468,2022-06-29,LS 주가 하락장서 역주행 모든 사업부가 호황,2022-06-30,-6.67,-1,1,1,0,1,주가 하락 장서 역주행 모든 사업 부가 호황,-1642.033698,-1
364469,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,-1,1,0,0,1,바사 토종 백신 호 식약처 승인 소식 강세,388.689351,1
364470,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,-1,0,1,0,-1,장 황 코스피 선 경기 침체 우려 약세,-832.257109,-1


In [60]:
df['sent_label'].value_counts()

 1    218300
-1    143064
 0      2969
Name: sent_label, dtype: int64

In [61]:
df['tokenized'] = 0

In [63]:
df.dropna(how = 'any', inplace = True)

# 형태소 분석
df['tokenized'] = df['Title'].apply(okt.morphs)  # 또는 자유롭게 라이브러리 사용

# X_train = df['tokenized'].values
# Y_train = df['sent_label'].values

# df.to_csv('./data/[Model4]tokenized_added_df.csv', encoding = 'utf-8-sig')

In [77]:
X_train = df['tokenized']
Y_train = df['sent_label']

In [78]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [79]:
# 빈도 수가 낮은 단어 제거
threshold = 3     # 3개 미만으로 등장하는 단어 제거

words_cnt = len(tokenizer.word_index)
rare_cnt = 0
words_freq = 0
rare_freq = 0

for key, value in tqdm(tokenizer.word_counts.items()) : 
    words_freq = words_freq + value

    if value < threshold : 
        rare_cnt += 1
        rare_freq += value

print("전체 단어 수 :", words_cnt)
print("빈도가 {} 이하인 희귀 단어 수 : {}".format(threshold-1, rare_cnt))
print("희귀 단어 비율 : {}".format((rare_cnt/words_cnt) * 100))
print("희귀 단어 등장 빈도 비율 : {}".format((rare_freq / words_freq) * 100))

100%|███████████████████████████████████████████████████████████████████████| 74145/74145 [00:00<00:00, 1183408.25it/s]

전체 단어 수 : 74145
빈도가 2 이하인 희귀 단어 수 : 38647
희귀 단어 비율 : 52.12354170881382
희귀 단어 등장 빈도 비율 : 1.2027451867060537


In [80]:
vocab_size = words_cnt - rare_cnt + 2
# vocab_size = words_cnt - rare_cnt     # +2 는 패딩, OOV???
print(vocab_size)

35500


In [81]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [82]:
print('리뷰 최대 길이 :', max(len(i) for i in X_train))

리뷰 최대 길이 : 42


In [83]:
# 패딩
max_len = max(len(i) for i in X_train)

X_train = pad_sequences(X_train, maxlen = max_len)

In [84]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train, 3)

#### 모델링-LSTM

In [75]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [86]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(100)))
# model.add(Dense(60))
model.add(Dense(3, activation = 'softmax'))

# val_loss를 관찰값으로 설정, 그 값이 감소하는 걸 멈출 때 학습 종료
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1, patience=5)

mc = ModelCheckpoint('2real_lstm_model4.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

model.summary()

Epoch 1/15
1139/1139 [==============================] - ETA: 0s - loss: 0.1572 - acc: 0.9403
Epoch 1: val_acc improved from -inf to 0.96963, saving model to 2real_lstm_model4.h5
1139/1139 [==============================] - 160s 137ms/step - loss: 0.1572 - acc: 0.9403 - val_loss: 0.0851 - val_acc: 0.9696
Epoch 2/15
1139/1139 [==============================] - ETA: 0s - loss: 0.0744 - acc: 0.9747
Epoch 2: val_acc improved from 0.96963 to 0.97512, saving model to 2real_lstm_model4.h5
1139/1139 [==============================] - 162s 142ms/step - loss: 0.0744 - acc: 0.9747 - val_loss: 0.0740 - val_acc: 0.9751
Epoch 3/15
1139/1139 [==============================] - ETA: 0s - loss: 0.0621 - acc: 0.9793
Epoch 3: val_acc improved from 0.97512 to 0.97572, saving model to 2real_lstm_model4.h5
1139/1139 [==============================] - 156s 137ms/step - loss: 0.0621 - acc: 0.9793 - val_loss: 0.0685 - val_acc: 0.9757
Epoch 4/15
1139/1139 [==============================] - ETA: 0s - loss: 0.0548 

In [96]:
Y_result = df['updown'].values

In [97]:
# 저장된 모델 불러오기, 평가
loaded_model = load_model('2real_lstm_model4.h5')
loaded_model.evaluate(X_train, Y_train)

11386/11386 [==============================] - 121s 10ms/step - loss: 0.0497 - acc: 0.9840


[0.04971154406666756, 0.9839679598808289]

In [99]:
loaded_model = load_model('2real_lstm_model4.h5')
loaded_model.evaluate(X_train, Y_result)   # 실제 값과 비교???

ValueError: in user code:

    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 1473, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\jinyo\.conda\envs\py123\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y

In [100]:
# 임의 불용어
stopwords1 = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords2 = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
stopwords = list(set(stopwords1 + stopwords2))

def sentiment_predict(new_sentence):
    new_token = [word for word in okt.morphs(new_sentence) if not word in stopwords]
    new_sequences = tokenizer.texts_to_sequences([new_token])
    new_pad = pad_sequences(new_sequences, maxlen = max_len)
    # score = float(loaded_model.predict(new_pad))
    score = loaded_model.predict(new_pad)

    if np.argmax(score) == 0 :
        print("{} -> 긍정 ({:.2f}%)\n".format(new_sentence, score.max() * 100))
    elif np.argmax(score) == 1 :
        print("{} -> 모름 ({:.2f}%)\n".format(new_sentence, score.max() * 100))
    elif np.argmax(score) == 2 :
        print("{} -> 부정 ({:.2f}%)\n".format(new_sentence, score.max() * 100))

In [101]:
sentiment_predict("[유럽개장] 장초반 상승세…英 1.03%↑")
sentiment_predict("[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감")
sentiment_predict("[이번주 증시] 경기침체 우려 지속…반등시 포트폴리오 재정비")
sentiment_predict("'2차전지 너마저'…인플레·경기침체에 장사 없나 [한경우의 케이스스터디]")
sentiment_predict("기관 매도 연일 증시 폭락")

[유럽개장] 장초반 상승세…英 1.03%↑ -> 모름 (99.96%)

[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감 -> 부정 (100.00%)

[이번주 증시] 경기침체 우려 지속…반등시 포트폴리오 재정비 -> 부정 (100.00%)

'2차전지 너마저'…인플레·경기침체에 장사 없나 [한경우의 케이스스터디] -> 부정 (99.95%)

기관 매도 연일 증시 폭락 -> 부정 (100.00%)

